In [ ]:
#Importing necessary libraries

import pandas as pd
import numpy as np
import shapely as shp
import scipy
from google.colab import files

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display


from sklearn.linear_model import LinearRegression
from shapely.geometry import LineString, Point

from scipy.stats import linregress
from scipy.optimize import curve_fit

In [ ]:
!pip install plotly

In [ ]:
file=files.upload()

Saving absorption.xlsx to absorption.xlsx


In [ ]:
df= pd.read_excel('absorption.xlsx', sheet_name='Sheet2')

In [ ]:
#df = df[df['wavelength'] > 350]
#df = df[df['wavelength'] < 700]

In [ ]:
df.shape

(1005, 6)

In [ ]:
#wavelength
wavelength=df['wavelength']

#absorption
sample1=df['sample1']
sample2=df['sample2']
sample3=df['sample3']
sample4=df['sample4']
sample5=df['sample5']

#numpy array
wavelengths =np.array(wavelength)
absorbance1 = np.array(sample1)
absorbance2 = np.array(sample2)
absorbance3 = np.array(sample3)
absorbance4 = np.array(sample4)
absorbance5 = np.array(sample5)

#Urbach energy

#function

In [ ]:
def first_part(wavelengths, absorption):
    # Constants for conversion
    h = 6.62607004e-34  # Planck's constant in J*s
    c = 2.998e8  # speed of light in m/s
    eV = 1.60218e-19  # energy of one electron volt in J

    # Convert wavelength to energy (in eV)
    energies = (h * c) / (np.array(wavelengths) * 1e-9) / eV  # Converts J to eV

    # Calculate the logarithm of the absorption coefficient
    ln_alpha = np.log(absorption)

    # dynamic plot using plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=energies, y=ln_alpha, mode='markers+lines', name='ln(alpha) vs. E'))

    # labels and title
    fig.update_layout(
        xaxis_title='Energy (eV)',
        yaxis_title='ln(alpha)',
        title='Dynamic Plot: Identifying the Linear Region'
    )

    # Add hover information
    fig.update_traces(hoverinfo='text', text=[f'E: {xi:.2f}, ln(alpha): {yi:.2f}' for xi, yi in zip(energies, ln_alpha)])

    fig.show()

In [ ]:

def second_part(wavelengths, absorption, n1, n2):
    # Filter out non-positive absorption values
    positive_indices = absorption > 0
    wavelengths = wavelengths[positive_indices]
    absorption = absorption[positive_indices]

    # Convert wavelength to energy (in eV)
    h = 6.62607004e-34  # Planck's constant in J*s
    c = 2.998e8  # speed of light in m/s
    eV = 1.60218e-19  # energy of one electron volt in J
    energies = (h * c) / (wavelengths * 1e-9) / eV

    # Calculate the logarithm of the absorption coefficient
    ln_alpha = np.log(absorption)

    # Select data within the specified energy range (n1-n2 eV)
    indices = np.where((energies >= n1) & (energies <= n2))[0]
    if len(indices) == 0:
        raise ValueError("No data found in the specified energy range")

    # Perform a linear fit within the n1-n2 eV range
    try:
        popt, _ = curve_fit(lambda x, m, c: m * x + c, energies[indices], ln_alpha[indices])
        slope, intercept = popt
        urbach_energy = 1 / slope
    except Exception as e:
        print("An error occurred during curve fitting:", e)
        urbach_energy = None

    #  dynamic plot using plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=energies,
        y=ln_alpha,
        mode='markers',
        marker=dict(size=2, color='blue', symbol='circle-open'),  # Modify size, color, and symbol here
        name='ln(alpha) vs E'
    ))

    # Add a line for the linear fit with modified properties
    if urbach_energy:
        fit_line = go.Scatter(
            x=energies[indices],
            y=energies[indices] * slope + intercept,
            mode='lines',
            line=dict(color='red', width=3),  # Modify color and width of the red line here
            name='Linear Fit'
        )
        fig.add_trace(fit_line)

    # labels and title
    fig.update_layout(
        xaxis_title='Energy (eV)',
        yaxis_title='ln(alpha)',
        title='ZnO Nanorads: Linear Fit for Urbach Energy ',
        width=1000,  # Set the figure width
        height=400  # Set the figure height
    )

    # Add hover information
    hover_data = [f'E: {xi:.2f}, ln(alpha): {yi:.2f}' for xi, yi in zip(energies, ln_alpha)]
    fig.update_traces(hoverinfo='text', text=hover_data)

    fig.show()

    # Display Urbach energy if available
    if urbach_energy:
        print(f"Urbach energy for ZnO nanorads: {urbach_energy:.4f} eV")
    else:
        print("No linear fit could be calculated.")

#sample1

In [ ]:
first_part(wavelengths, absorbance1)

<ipython-input-76-2b9d9c3d326f>:11: RuntimeWarning:

invalid value encountered in log



In [ ]:
second_part(wavelengths, absorbance1, 2.01, 2.3)

Urbach energy for ZnO nanorads: 0.1665 eV


#sample2

In [ ]:
first_part(wavelengths, absorbance2)

In [ ]:
second_part(wavelengths, absorbance2, 2.06, 2.35)

Urbach energy for ZnO nanorads: 0.3441 eV


#sample3

In [ ]:
first_part(wavelengths, absorbance3)

In [ ]:
second_part(wavelengths, absorbance3, 2.11, 2.46)

Urbach energy for ZnO nanorads: 0.3123 eV


#sample4

In [ ]:
first_part(wavelengths, absorbance4)

In [ ]:
second_part(wavelengths, absorbance4, 2.11, 2.50)

Urbach energy for ZnO nanorads: 0.2792 eV


#sample5

In [ ]:
first_part(wavelengths, absorbance5)

In [ ]:
second_part(wavelengths, absorbance5, 2.11, 2.50)

Urbach energy for ZnO nanorads: 0.3334 eV


#millielectron volts (meV)


---



*   sample1:    166.5
*   sample2:    344.1
*   sample3:    312.3
*   sample4:    279.2
*   sample5:    333.4






---



#🔍

The Beer-Lambert law states that the intensity of light absorbed by a material is proportional to the concentration of the absorbing species and the path length of the light through the material. Mathematically, it can be expressed as:

A = α * c * d

where:

*   A is the absorbance
*   α is the absorption coefficient
*   d is the path length of light through the material
*   c is the concentration of the absorbing species


To determine the absorption coefficient α, you need to measure the absorbance A at different wavelengths and calculate α using the equation:

α = A / (c * d)

#https://gepac.github.io/2019-06-07-projeto-bandGap/

α=2.303A/L

L comes from the sample size